In [1]:
import google.generativeai as genai
import pandas as pd

In [2]:
def get_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

api_key_file = "./API_Key/Gemini.txt"  
api_key = get_api_key(api_key_file)
genai.configure(api_key=api_key)

model = genai.GenerativeModel("models/gemini-1.5-pro")

In [3]:
def configure_genai(api_key_file):
    with open(api_key_file, "r") as file:
        api_key = file.read().strip()
    genai.configure(api_key=api_key)
    return genai.GenerativeModel("gemini-1.5-pro")  

In [4]:
def generate_ner_tags(sentence, model):
    prompt = f"""
    You are a highly intelligent language model trained for natural language understanding and named entity recognition. 
    Your task is to assign the most suitable ner_tags to every token in the following text based on the provided list of possible ner_tags.

    Text:
    {sentence}

    Possible ner_tags:
    ['B-Further Specification', 'I-Further Specification', 'O', 'I-XOR Gateway', 'B-XOR Gateway', 'B-Activity Data', 'B-Condition Specification', 'I-Actor', 'B-AND Gateway', 'I-Activity', 'B-Activity', 'I-Activity Data', 'I-Condition Specification', 'I-AND Gateway', 'B-Actor']

    Your output must assign a ner_tag to each token in the same order as they appear in the input text. Use the following format for your output:

    Output:
    A Python-style list of ner_tags corresponding to the tokens in the same sequence. Example:
    ['O', 'O', 'O', 'O', 'B-Actor', 'I-Actor', 'O', ...]

    Be concise and only output the final list of ner_tags without any additional explanations.
    """
    response = model.generate_content(prompt)
    return response.text.strip()

In [5]:
def process_dataset(input_file, output_file, api_key_file):
    model = configure_genai(api_key_file)

    dataset = pd.read_csv(input_file)

    def process_row(row):
        sentence = " ".join(eval(row))  
        try:
            return generate_ner_tags(sentence, model)
        except Exception as e:
            return f"Error: {str(e)}"

    dataset["Gemini-1.5-pro"] = dataset["tokens"].apply(process_row)

    dataset.to_csv(output_file, index=False)
    print(f"Updated dataset saved to {output_file}")

In [6]:
input_file = "PET-Customized.csv" 
output_file = "PET-Customized-gemini-1.5-pro.csv" 
api_key_file = "./API_Key/Gemini.txt"

process_dataset(input_file, output_file, api_key_file)

Updated dataset saved to PET-Customized-gemini-1.5-pro.csv
